# NASA MODIS green data

In [81]:
# Initialize Earth Engine
import ee
ee.Initialize()

In [82]:
import geemap

# Define an area of interest (optional). Here, we'll use the entire world.
AOI = ee.Geometry.Rectangle([-180, -90, 180, 90])

# Fetch the MODIS NDVI dataset for a given date range
start_date = '2000-06-01'
end_date = '2000-07-01'
MODIS_NDVI = ee.ImageCollection('MODIS/061/MOD13A2').filterDate(ee.Date(start_date), ee.Date(end_date)).select('NDVI')

# Calculate the mean NDVI for the date range
mean_NDVI = MODIS_NDVI.mean()

# Define visualization parameters
vis_params = {
    'min': 0,
    'max': 9000,
    'palette': ['red', 'yellow', 'green', 'darkgreen', 'blue'],
    'breakpoints': [0, 3000, 6000, 8500, 9000]
}

# Create a map and add the NDVI layer
Map = geemap.Map()
Map.centerObject(AOI, 2)  # Center the map to our area of interest and set zoom level
Map.addLayer(mean_NDVI, vis_params, 'Mean NDVI')
Map.addLayerControl()  # This will add a layer control to toggle layers
Map.add_colorbar(vis_params, label="NDVI")
Map



Map(center=[4.075891298132545e-31, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=…

In [83]:
import geemap

# Define an area of interest (optional). Here, we'll use the entire world.
AOI = ee.Geometry.Rectangle([-180, -90, 180, 90])

# Fetch the MODIS NDVI dataset for a given date range
start_date = '2022-06-01'
end_date = '2022-07-01'
MODIS_NDVI = ee.ImageCollection('MODIS/061/MOD13A2').filterDate(ee.Date(start_date), ee.Date(end_date)).select('NDVI')

# Calculate the mean NDVI for the date range
mean_NDVI = MODIS_NDVI.mean()

# Define visualization parameters
vis_params = {
    'min': 0,
    'max': 9000,
    'palette': ['red', 'yellow', 'green', 'darkgreen', 'blue'],
    'breakpoints': [0, 3000, 6000, 8500, 9000]
}

# Create a map and add the NDVI layer
Map = geemap.Map()
Map.centerObject(AOI, 2)  # Center the map to our area of interest and set zoom level
Map.addLayer(mean_NDVI, vis_params, 'Mean NDVI')
Map.addLayerControl()  # This will add a layer control to toggle layers
Map.add_colorbar(vis_params, label="NDVI")
Map


Map(center=[4.075891298132545e-31, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=…

# Look at difference between 2000 and 2022


In [84]:
# Define date ranges for summers of 2000-2003 and 2019-2022
dates_2000_2003 = [('2000-06-01', '2000-08-31'), ('2001-06-01', '2001-08-31'), 
                   ('2002-06-01', '2002-08-31'), ('2003-06-01', '2003-08-31')]

dates_2019_2022 = [('2019-06-01', '2019-08-31'), ('2020-06-01', '2020-08-31'), 
                   ('2021-06-01', '2021-08-31'), ('2022-06-01', '2022-08-31')]

# Fetch and average MODIS NDVI data for each period
def fetch_avg_NDVI(dates):
    image_list = []
    for start, end in dates:
        print(f'Fetching data for {start} to {end}')

        # Fetch the MODIS NDVI dataset for a given date range
        MODIS_NDVI = ee.ImageCollection('MODIS/061/MOD13A2').filterDate(ee.Date(start), ee.Date(end)).select('NDVI')

        # Calculate the mean NDVI for the date range
        mean_NDVI = MODIS_NDVI.mean()
       
        # Add the image to the list
        image_list.append(mean_NDVI)

    # Combine the list into an image collection
    print(f'\nlength of image_list: {len(image_list)}')
    combined_collection = ee.ImageCollection(image_list)

    return combined_collection.mean()

avg_NDVI_2000_2003 = fetch_avg_NDVI(dates_2000_2003)
avg_NDVI_2019_2022 = fetch_avg_NDVI(dates_2019_2022)



Fetching data for 2000-06-01 to 2000-08-31
Fetching data for 2001-06-01 to 2001-08-31
Fetching data for 2002-06-01 to 2002-08-31
Fetching data for 2003-06-01 to 2003-08-31

length of image_list: 4
Fetching data for 2019-06-01 to 2019-08-31
Fetching data for 2020-06-01 to 2020-08-31
Fetching data for 2021-06-01 to 2021-08-31
Fetching data for 2022-06-01 to 2022-08-31

length of image_list: 4


# ... Visualization and further analysis ...


In [85]:
# ... Visualization and further analysis ...

# Compute the difference between the two periods
NDVI_difference = avg_NDVI_2019_2022.subtract(avg_NDVI_2000_2003)

# Define visualization parameters
vis_params = {
    'min': -9000,
    'max': 9000,
    'palette': ['darkred', 'red', 'lightcoral', 'white', 'lightgreen', 'green', 'darkgreen']


}


# Create a map and add the NDVI layer
Map = geemap.Map()
Map.centerObject(AOI, 2)  # Center the map to our area of interest and set zoom level
Map.addLayer(mean_NDVI, vis_params, 'Mean NDVI')
Map.addLayerControl()  # This will add a layer control to toggle layers
Map.add_colorbar(vis_params, label="NDVI")

Map

Map(center=[4.075891298132545e-31, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=…

In [86]:
print(NDVI_difference.getInfo())

{'type': 'Image', 'bands': [{'id': 'NDVI', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -65535, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


# Sampling & Clustering

In [87]:
# Define the region of interest as Africa
region = ee.Geometry.Rectangle([-20, -35, 55, 37])



# Define the number of clusters
n_clusters = 5


# Sample the image to generate a FeatureCollection of points
sample = NDVI_difference.sample({
    'region': region,
    'scale': 1000,
    'numPixels': 10000
})

# Train the clusterer using the sampled points
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(sample)
result = NDVI_difference.cluster(clusterer)

# Define a color palette
palette = ['red', 'orange', 'yellow', 'lime', 'green']

# Add the classified image to the map
Map.addLayer(result, {'min': 0, 'max': n_clusters - 1, 'palette': palette}, 'Clusters')

Map


EEException: Invalid GeoJSON geometry.